# Programmation d'un modèle de langue n-gram

# Import des packages

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np

from ngram import NGramModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Traitement des données d'apprentissage

On utilise ici l'extrait assez faible (15 Mo) disponible sur le drive de Benjamin.

# Tests
## Greedy prediction

In [23]:
with open("fr_wikipedia_sample.txt","r",encoding="utf8") as file:
    corpus=file.read()
corpus = corpus.lower()
seqcorpus = corpus.split(' ')
    
for i in range(2,6):
    # print("Résultats pour",i,"gram")
    model = NGramModel(i)
    model.fit(seqcorpus)
    print("Predicting...")
    text = " ".join(model.generate_greedy(50,"elle est la")))
    print(text)
    print("")

Fitting 2-gram model on vocabulary of size 133657.
Predicting...
('elle', 'est', 'la', 'première', 'fois', ',', 'le', 'plus', 'de', 'la', 'première', 'fois', ',', 'le', 'plus', 'de', 'la', 'première', 'fois', ',', 'le', 'plus', 'de', 'la', 'première', 'fois', ',', 'le', 'plus', 'de', 'la', 'première', 'fois', ',', 'le', 'plus', 'de', 'la', 'première', 'fois', ',', 'le', 'plus', 'de', 'la', 'première', 'fois', ',', 'le', 'plus', 'de', 'la', 'première')

Fitting 3-gram model on vocabulary of size 133657.
Predicting...
('elle', 'est', 'la', 'plus', 'grande', 'partie', 'de', 'la', 'population', 'de', "l'", 'état', 'de', 'la', 'population', 'de', "l'", 'état', 'de', 'la', 'population', 'de', "l'", 'état', 'de', 'la', 'population', 'de', "l'", 'état', 'de', 'la', 'population', 'de', "l'", 'état', 'de', 'la', 'population', 'de', "l'", 'état', 'de', 'la', 'population', 'de', "l'", 'état', 'de', 'la', 'population', 'de', "l'")

Fitting 4-gram model on vocabulary of size 133657.
Predicting...


KeyboardInterrupt: 

## Beam Search

Remarque : ça se met souvent à boucler ! Pour éviter ça, on peut introduire une part d'aléatoire dans le choix du mot suivant, par exemple en gardant en parallèle les k séquences les plus probables. Cette méthode s'appelle le beam search.

In [123]:
#Génère une séquence de nbmots mots conditionnellement à la séquence précédente seqprec
#Avec la méthode Beam Search pour k meilleurs séquences conservées à chaque étape
def genererbeam(nbmots,seqprec,k):
    sequences=[[seqprec,1]]
    #A chaque etape
    for i in range(nbmots):
        #calcule tous les candidats possibles de l'étape (sequence totale et proba associée)
        candidates=[]
        for j in range(len(sequences)):
            seq, prob = sequences[j]
            for mot in vocabulaire:
                candidates.append([seq+" "+mot,prob*probacond(mot," ".join(seq.split(' ')[-(n-1):]))])
        # ordonne les candidats selon le score
        ordered = sorted(candidates, key=lambda tup:tup[1])
        # sélectionne les k meilleurs
        sequences = ordered[-k:]
    return sequences

In [104]:
#Cas bigram
genererbeam(30,"<s>",3)

[['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la ville de la ville',
  7.486873168181089e-34],
 ['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la ville de la première',
  8.241045373767097e-34],
 ['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la première fois , le',
  9.530640308839064e-34]]

In [116]:
#Cas trigram
genererbeam(30,"<s> il",3)

[["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> la première",
  2.592672590660988e-27],
 ["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays",
  2.841209628240636e-27],
 ["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le premier",
  3.1430881512412038e-27]]

In [124]:
#Cas quatrigram
genererbeam(50,"<s> il est",3)

[["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie grâce aux",
  2.0868169263891186e-11],
 ["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie , par",
  2.608521157986398e-11],
 ["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie grâce à",
  8.347267705556474e-11]]

In [ ]:
#Cas 5-gram
genererbeam(30,"<s> il est le",3)

Améliorations :
-> lissage (smoothing)
-> Apprendre les fréquences de tous les k-grams pour k<n pour pouvoir switcher à un k plus petit si le k-gram recherché est absent lors de la prédiction
-> Travailler avec les log-probas pour être sûr de ne pas perdre en précision -> Le temps pour générer le texte me semble très long, il y a sans doute moyen d'optimiser le code